## Ingesting Book....

In [2]:
from typing import Iterator
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document as LCDocument
from docling.document_converter import DocumentConverter

class DoclingPDFLoader(BaseLoader):
    def __init__(self, file_path: str | list[str]) -> None:
        self._file_paths = file_path if isinstance(file_path, list) else [file_path]
        self._converter = DocumentConverter()

    def lazy_load(self) -> Iterator[LCDocument]:
        for source in self._file_paths:
            dl_doc = self._converter.convert(source).document
            text = dl_doc.export_to_markdown()
            yield LCDocument(page_content=text)

In [3]:
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

C:\Users\noeay\AppData\Local\Temp\ipykernel_7844\2807142782.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [7]:
local_path = "./data/canguilhem.pdf"

# Local PDF file uploads
if local_path:
  loader = DoclingPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

C:\Users\noeay\AppData\Roaming\Python\Python311\site-packages\torch\utils\cpp_extension.py:414: UserWarning: Error checking compiler version for cl: [WinError 2] Le fichier spécifié est introuvable
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')
C:\Users\noeay\AppData\Roaming\Python\Python311\site-packages\torch\utils\cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Could not load the custom kernel for multi-scale deformable attention: Command '['where', 'cl']' returned non-zero exit status 1.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: Le module spécifié est introuvable.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: Le

In [8]:
# Preview first page
Markdown(data[0].page_content)

'.\n\n.\'\n\n## · SCIENCE ET PENSEE\n\n•\n\nN os gestes, nos actes, notre vie dependent des decou› vertes scientifiques. Mais, de ces decouvertes, nous ignorons souvent les voies, le contenu, le sens : chacun est maintenant depasse par l~ Savoir humain comme chacun \'1\' etait hier par la N atu1\'e, et beaucoup eprouvent aujour› d\'hui, a l\'egard de la science, les sentiments que jadis inspiraient les choses : peur panique ou 1\'eligieuse admim› tion.\n\nF. ALQUIE.\n\nLa collection « Science et Pensee », qemandant ades chercheu1\'s authentiques d\'exposer aun public non spe› cialise les methodes et les 1\'eSUltats de la science contempo› raine, voudrait favoriser la prise de conscience de ce Savoir qui donne anotre monde son visage. Est-il, en elfet, pour l\'homme moderne, tache pltts u1\'gente que de s\'elever ala ha..uteur de son pmpre esprit ?\n\n- GABRIEL-REY. HUMANISME ET SURHU› MANISME\n- Dr. NEGRE. L\'HOMME ET LA BOMBE. Les effets biologiques de la bombe atomique.\n- RENE NELLI

## Vector Embeddings

In [ ]:
!ollama list

In [ ]:
# # Pull nomic-embed-text model from Ollama if you don't have it
# !ollama pull nomic-embed-text
# # List models again to confirm it's available
# !ollama list

In [5]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [6]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

NameError: name 'data' is not defined

In [ ]:

# 3. Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# 4. Now reimport with the new versions
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# 5. Try creating the vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="mxbai-embed-large"),
    collection_name="local-rag"
)

## Retrieval

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
!ollama list

In [ ]:
# LLM from Ollama
local_model = "llama3.2"
llm = ChatOllama(model=local_model)

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [ ]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("What are the 5 pillars of global cooperation?")

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()